In [ ]:
from pathlib import Path

import cartopy.crs as ccrs
import earthaccess
import matplotlib.pyplot as plt
import numpy as np
import requests, pdb
import xarray as xr

auth = earthaccess.login(persist=True)
fs = earthaccess.get_fsspec_https_session()

tspan = ("2024-12-01", "2025-01-31")
#tspan = ("2025-01-16", "2025-01-31")

#bbox = (-65.44, 8.03, -5.06, 38.75) # W, S, E, N
#bbox = (-129.5, 27, -120, 29) # west, south, east, north
bbox = (-140.5, 10, -100, 39)  # west, south, east, north
bbox = (-180, 0, -100, 39)  # west, south, east, north

clouds = (0, 100) #cloud cover 0-50%

#===============================================
#1. get lists of timestamps that sweeps thru our coordinates

#2. make a for loop for all the timestamps and download oci aod (because oci aod is not offered in earthaccess, we cannot designate bbox)
#===============================================

#1.

results = earthaccess.search_data(
    short_name="PACE_OCI_L2_BGC", #example var.
    temporal=tspan,
    bounding_box=bbox,
    cloud_cover=clouds,
)
# (9 24 25)
tstamps = [] #lists of timestamps
for i in range(len(results)):
    fname = results[i]['meta']['native-id']
    rec = fname.strip().split('.')
    tstamps.append(rec[1])
    
#2.
OB_DAAC_PROVISIONAL = "https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/"
for tstamp in tstamps:
    
    HARP2_L2_MAPOL_FILENAME = "PACE_OCI.{}.L2.AER_UAA.V3_0.NRT.nc".format(tstamp)
    try:
        fs.get(f"{OB_DAAC_PROVISIONAL}/{HARP2_L2_MAPOL_FILENAME}", "/data/FirePhytos_DustBlumes/PACE_OCI_AOD/Pacific/")
        print(tstamp)
    except:
        pass
    # paths = list(Path("data").glob("*.nc"))
    # for path in paths:
    #     reader = [
    # pdb.set_trace()

20241207T004616
20241207T005116
20241207T005616
20241207T010116
20241207T185220
20241207T185720
20241207T190220
20241207T202537
20241207T203037
20241207T203537
20250117T211622
20250117T212122
20250118T002800
20250118T003300


In [13]:
from pathlib import Path

import cartopy.crs as ccrs
import earthaccess
import matplotlib.pyplot as plt
import numpy as np
import requests
import xarray as xr
from glob import glob
from datetime import datetime
import pdb

def plot_l2_product(
    lon, lat, data, plot_range, label, title, vmin, vmax,fn, figsize=(12, 4), cmap="jet"):
    """Make map and histogram (default)."""

    # Create a figure with two subplots: 1 for map, 1 for histogram
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(1, 2, width_ratios=[3, 1], wspace=0.3)

    # Map subplot
    ax_map = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
    ax_map.set_extent(plot_range, crs=ccrs.PlateCarree())
    ax_map.coastlines(resolution="110m", color="black", linewidth=0.8)
    ax_map.gridlines(draw_labels=True)
    ax_map.set_title(title, fontsize=12)

    # Assume lon and lat are defined globally or passed in
    for i in range(len(data)):
        pm = ax_map.pcolormesh(
            lon[i], lat[i], data[i], vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree(), cmap=cmap
        )
    plt.colorbar(pm, ax=ax_map, orientation="vertical", pad=0.1, label=label)


    # Histogram subplot
    # ax_hist = fig.add_subplot(gs[1])
    # flattened_data = data[~np.isnan(data)]  # Remove NaNs for histogram
    # valid_count = np.sum(~np.isnan(flattened_data))
    # ax_hist.hist(
    #     flattened_data, bins=40, color="gray", range=[vmin, vmax], edgecolor="black"
    # )
    # ax_hist.set_xlabel(label)
    # ax_hist.set_ylabel("Count")
    # ax_hist.set_title("Histogram: N=" + str(valid_count))

    plt.tight_layout()
    plt.savefig(fn)
    plt.clf()

target_date = datetime(2025,1,5)
oci_dir = '/home/jovyan/shared-public/FirePhytos_DustBlumes/PACE_OCI_AOD/'
files = sorted(glob(oci_dir + target_date.strftime('*%Y%m%dT*')))
print(len(files))
# pdb.set_trace()

aods = []
ssas = []
lons = []
lats = []

for file in files:

    datatree = xr.open_datatree(file)
    dataset = xr.merge(datatree.to_dict().values())
    lon = np.array(dataset['longitude'])
    lat = np.array(dataset['latitude'])
    aod = np.squeeze(np.array(dataset['Aerosol_Optical_Depth'][:,:,3]))
    # fmf = np.squeeze(np.array(dataset['Aerosol_Optical_Depth'][:,:,3]))
    ssa_388_dt = np.squeeze(np.array(dataset['DT_AerosolSingleScattAlbedo'][:,:,1]))
    ssa_388_nuv = np.squeeze(np.array(dataset['NUV_AerosolSingleScattAlbedo'][:,:,1]))

    lons.append(lon)
    lats.append(lat)
    aods.append(aod)
    ssas.append(ssa_388_nuv)
    
    
plot_range = [np.nanmin(lons), np.nanmax(lons), np.nanmin(lats), np.nanmax(lats)]
    
plot_l2_product(lons, lats,
aods, plot_range=plot_range, label='', title=target_date.strftime('AOD_%Y%m%d'), vmin=0, vmax=2, 
                fn=target_date.strftime('PACE_OCI_AOD_%Y%m%d.png'), cmap="jet")

# plot_l2_product(lon, lat,
# ssa_388_dt, plot_range=plot_range, label='', title='DT_SSA', vmin=0.7, vmax=1, 
#                 fn=target_date.strftime('PACE_OCI_DTSSA_%Y%m%d.png'), cmap="jet")

plot_l2_product(lons, lats,
ssas, plot_range=plot_range, label='', title='NUV_SSA', vmin=0.7, vmax=1, 
                fn=target_date.strftime('PACE_OCI_NUVSSA_%Y%m%d.png'), cmap="jet")
    

8


/tmp/ipykernel_110/1800623467.py:47: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/tmp/ipykernel_110/1800623467.py:47: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


<Figure size 1200x400 with 0 Axes>

<Figure size 1200x400 with 0 Axes>